In [1]:
from uncertainty_analysis import multilayer_perceptron_ensemble as mpe
import pandas as pd

In [2]:
import openml

dataset = openml.datasets.get_dataset(44061)
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

C:\Users\elsag\OneDrive\Desktop\Uni\Year 4\Academic\Dissertation\Final Report\Code\scikit-learn-1.2.1\scikit-learn-1.2.1\venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [3]:
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
columns_to_encode = ['X3', 'X4', 'X6']
X = pd.get_dummies(X, columns=columns_to_encode)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
X_train.head()

,X10,X12,X13,X14,X15,X16,X17,X18,X19,X20,...,X6_10,X6_11,X6_2,X6_3,X6_4,X6_5,X6_6,X6_7,X6_8,X6_9
370,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3392,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2208,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3942,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1105,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
X_train.shape

(2946, 379)

In [7]:
X_test.shape

(1263, 379)

In [8]:
merc_nn_ens = []
merc_nn_ens_preds = [] # List[ensemble np.array[regressors np.array[predictions]]
for i in range(0, 10):
    m = mpe.MultiMLPRegressors(n_jobs=10, n_regressors=100, alpha=0.0001, hidden_layer_sizes=(100, 150, 100), activation='relu', max_iter=500, solver='adam').fit(X_train, y_train)
    merc_nn_ens.append(m)
    merc_nn_ens_preds.append(m.predict(X_test.apply(pd.to_numeric)))

In [9]:
from uncertainty_analysis import project_helper_functions as phf

best_model_results = merc_nn_ens_preds[phf.get_best_model_idx(merc_nn_ens_preds, y_test)]
phf.write_ensemble_model_results(best_model_results.T, "merc_ensemble_nn_results.txt")